# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given video or an image.



## Install OpenPose

In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

Selecting previously unselected package libgflags2.2.
(Reading database ... 135718 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

## Mount Google drive

We used google drive for data storage 

In [0]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

# List of all filesnames
files = os.listdir(path='/content/drive/My Drive/data-x_videos_2')
# Get list of lables
files = [x.replace('.mp4','') for x in files]
print(files)

['balbir_1', 'balbir_2', 'balbir_3', 'balbir_4', 'mayank_1', 'mayank_2', 'mayank_3', 'hari_1', 'ojas_1', 'ojas_2', 'ojas_3', 'ojas_4', 'ojas_5', 'suri_1', 'sunit_1', 'suri_3', 'suri_4', 'sunit_2', 'suri_5', 'sunit_3', 'sunit_4', 'sunit_5', 'hari_2', 'balbir_5', 'hari_3', 'hari_4', 'hari_5', 'sahej_1', 'mayank_4', 'sahej_2', 'mayank_5', 'sahej_3', 'mayank_6', 'sahej_4', 'sahej_5', 'suneel_1', 'manish_1', 'suneel_2', 'manish_2', 'manish_3', 'suneel_3', 'manish_4', 'suneel_4', 'manish_5', 'suneel_5', 'tushar_1', 'karthika_1', 'tushar_2', 'karthika_2', 'karthika_3', 'tushar_3', 'plaksha_4', 'tushar_4', 'karthika_5', 'shray_1', 'shray_2', 'na_1', 'shray_3', 'na_2', 'shray_4', 'na_3', 'na_4', 'na_5', 'ribhu_1', 'nb_1', 'ribhu_2', 'nb_2', 'ribhu_3', 'ribhu_4', 'nb_3', 'ribhu_5', 'nb_4', 'nb_5', 'nc_1', 'nc_2', 'nc_3', 'nc_4', 'nc_5', 'nd_1', 'nd_2', 'urvin_1', 'urvin_2', 'nd_3', 'urvin_3', 'nd_4', 'nd_5', 'hemant_1', 'ne_1', 'hemant_2', 'ne_2', 'hemant_3', 'ne_3', 'ne_4', 'hemant_4', 'ne_5', 

Download the above youtube video, cut the first 5 seconds and do the pose detection on that 5 seconds:

In [0]:
from tqdm import tqdm_notebook as tqdm

In [0]:
data_path = "/content/drive/My\ Drive/data-x_videos_2/"
json_path = "/content/drive/My\ Drive/data-x_json_2/"

# Loop over all video files
for vid in tqdm(files):
  # 1. Cut the first 5 seconds:
  !ffmpeg -y -loglevel info -i {data_path + vid + '.mp4'} -t 5 video.mp4
  
  # 2. Detect poses on the these 5 seconds:
 
  # Path 
  path = os.path.join("/content/drive/My Drive/data-x_json_2/", vid)
  print('************************************************************')
  print(path) 
  # Create output directory to store json of keypoints
  os.mkdir(path) 
  !rm openpose.avi
  # Generate json of keypoints
  !cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 --write_json {json_path+vid+'/'} --display 0 --write_video ../openpose.avi
  #convert the result into MP4
  #!ffmpeg -y -loglevel info -i openpose.avi output.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

Finally, visualize the result:

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=960, height=720)